# Session Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

import json
import unittest

from common import testExit, projdir

from entrant import Entrant
from period import Period
from course import Course

from constants import *

## Session Class

Class to manage sessions

In [2]:
class Session(Period):
    def __init__(self, event, dataPath, configPath, verbosity=1):
        '''Initialise session object'''

        super().__init__(parent=event, verbosity=verbosity)

        self.dataPath = dataPath
        self.configPath = configPath

        self.date = os.path.basename(configPath)
        self.year = self.date[:4]

        self.courses = {}


    def loadRuns(self, dataDir):
        '''Read speeds from CSV files'''

        filename = os.path.join(self.configPath, COURSES_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            try:
                config = json.loads(jsonTxt)
            except:
                self.logError('Could not parse {}'.format(filename))
                raise

        csvPaths = sorted(glob.glob(os.path.join(self.dataPath, '*')))
        
        for csvPath in csvPaths:
            suffix = os.path.splitext(csvPath)[1]
            fileParts = os.path.splitext(os.path.basename(csvPath))[0].split('_')

            if len(fileParts) >= 3:
                prefix, sessionDate, courseId = fileParts[:3]
            elif len(fileParts) == 2:
                prefix, sessionDate = fileParts
                courseId = COURSE_ID_TBC
            else:
                raise ValueError('Invalid filename {}'.format(os.path.basename(csvPath)))
            
            # Verify the filename
            if suffix.lower() != '.csv':
                raise ValueError('Invalid file suffix "{}" for {}'.format(suffix, os.path.basename(csvPath)))
            if prefix.upper() != dataDir.upper():
                raise ValueError('Invalid file prefix "{}" for {}'.format(prefix, os.path.basename(csvPath)))
            if sessionDate != self.date:
                raise ValueError('Invalid file date "{}" for {}'.format(sessionDate, os.path.basename(csvPath)))
            if courseId in self.courses:
                raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))

            startTime = endTime = None
            distance = 500

            if DEFAULT_COURSE in config:
                if T_START_TIME in config[DEFAULT_COURSE]:
                    startTime = config[DEFAULT_COURSE][T_START_TIME]
                if T_END_TIME in config[DEFAULT_COURSE]:
                    endTime = config[DEFAULT_COURSE][T_END_TIME]

            if courseId in config:
                if T_START_TIME in config[courseId]:
                    startTime = config[courseId][T_START_TIME]
                if T_END_TIME in config[courseId]:
                    endTime = config[courseId][T_END_TIME]
                if T_DISTANCE in config[courseId]:
                    distance = config[courseId][T_DISTANCE]

            if startTime is None or endTime is None:
                raise ValueError('Course times unavailable for {}'.format(os.path.basename(csvPath)))

            course = Course(self, courseId, startTime, endTime, distance=distance, verbosity=self.verbosity)
            self.courses[courseId] = course
            course.loadRuns(csvPath)

        self.sortRuns()

## Unit Tests

A handful of very basic tests, including a dummy event class

In [3]:
class DummyEvent(Period):
    def __init__(self, path, verbosity=1):
        '''Initialise event object'''

        super().__init__()

        self.path = path
        
        self.appConfig = appConfig


    def loadConfig(self):
        '''Read app config from JSON'''

        filename = os.path.join(self.path, CONFIG_DIR, EVENT_CONFIG)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            self.eventConfig = json.loads(jsonTxt)


    def seedEntrants(self):
        '''Read app config from JSON'''

        self.entrants[0] = Entrant(self.eventConfig)

In [4]:
class TestSession2000(unittest.TestCase):
    '''Class to test Session class'''
    
    def test20001003(self, event=None):
        '''Test using 20001003 data'''

        sessionDate = '20001003'
        eventYear = sessionDate[:4]

        if event is None:
            eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
            event = DummyEvent(eventPath)
            event.loadConfig()
            event.seedEntrants()

        dataPath = os.path.join(projdir, EVENTS_DIR, eventYear, RUNDATA_DIR, sessionDate)
        configPath = os.path.join(projdir, EVENTS_DIR, eventYear, SESSIONS_DIR, sessionDate)
        
        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        session = Session(event, dataPath, configPath, verbosity=0)
        session.loadRuns(RUNDATA_DIR)

        self.assertEqual(session.numRuns, 370)
        self.assertEqual(len(session.runs), 30)

        # Check runs are sorted correctly
        for personId in session.runs:
            maxSpeed = 99.999
            for run in session.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def test20001005(self, event=None):
        '''Test using 20001005 data'''

        sessionDate = '20001005'
        eventYear = sessionDate[:4]

        if event is None:
            eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
            event = DummyEvent(eventPath)
            event.loadConfig()
            event.seedEntrants()

        dataPath = os.path.join(projdir, EVENTS_DIR, eventYear, RUNDATA_DIR, sessionDate)
        configPath = os.path.join(projdir, EVENTS_DIR, eventYear, SESSIONS_DIR, sessionDate)
        
        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        session = Session(event, dataPath, configPath, verbosity=0)
        session.loadRuns(RUNDATA_DIR)

        self.assertEqual(session.numRuns, 247)
        self.assertEqual(len(session.runs), 21)

        # Check runs are sorted correctly
        for personId in session.runs:
            maxSpeed = 99.999
            for run in session.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def test201910(self, event=None):
        '''Test using 20001003 + 20001005 data'''

        # Dummy event will be shared for two sessions
        eventYear = '2000'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
        event = DummyEvent(eventPath)
        event.loadConfig()
        event.seedEntrants()

        # Load two sessions
        self.test20001003(event=event)
        self.test20001005(event=event)

        # Check the event totals
        self.assertEqual(event.numRuns, 617)
        self.assertEqual(len(event.runs), 35)

In [5]:
class TestSession2019(unittest.TestCase):
    '''Class to test Session class'''
    
    def test20191010(self, event=None):
        '''Test using 20191010 data'''

        sessionDate = '20191010'
        eventYear = sessionDate[:4]

        if event is None:
            eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
            event = DummyEvent(eventPath)
            event.loadConfig()
            event.seedEntrants()

        dataPath = os.path.join(projdir, EVENTS_DIR, eventYear, GPSDATA_DIR, sessionDate)
        configPath = os.path.join(projdir, EVENTS_DIR, eventYear, SESSIONS_DIR, sessionDate)
        
        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        session = Session(event, dataPath, configPath, verbosity=0)
        session.loadRuns(GPSDATA_DIR)

        self.assertEqual(session.numRuns, 638)
        self.assertEqual(len(session.runs), 40)

        # Check runs are sorted correctly
        for personId in session.runs:
            maxSpeed = 99.999
            for run in session.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def test20191011(self, event=None):
        '''Test using 20191011 data'''

        sessionDate = '20191011'
        eventYear = sessionDate[:4]

        if event is None:
            eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
            event = DummyEvent(eventPath)
            event.loadConfig()
            event.seedEntrants()

        dataPath = os.path.join(projdir, EVENTS_DIR, eventYear, GPSDATA_DIR, sessionDate)
        configPath = os.path.join(projdir, EVENTS_DIR, eventYear, SESSIONS_DIR, sessionDate)
        
        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        session = Session(event, dataPath, configPath, verbosity=0)
        session.loadRuns(GPSDATA_DIR)

        self.assertEqual(session.numRuns, 515)
        self.assertEqual(len(session.runs), 36)

        # Check runs are sorted correctly
        for personId in session.runs:
            maxSpeed = 99.999
            for run in session.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def test201910(self, event=None):
        '''Test using 20191010 + 20191011 data'''

        # Dummy event will be shared for two sessions
        eventYear = '2019'
        eventPath = os.path.join(projdir, EVENTS_DIR, eventYear)
        event = DummyEvent(eventPath)
        event.loadConfig()
        event.seedEntrants()

        # Load two sessions
        self.test20191010(event=event)
        self.test20191011(event=event)

        # Check the event totals
        self.assertEqual(event.numRuns, 1153)
        self.assertEqual(len(event.runs), 47)

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [6]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

......
----------------------------------------------------------------------
Ran 6 tests in 0.195s

OK


## All Done!